In [1]:
import g4f
from time import sleep
import numpy as np
from fuzzywuzzy import fuzz

In [ ]:
TEMPLATE = """Brainstorm a list of potentially useful text retrieval tasks.
Here are a few examples for your reference:
{demonstration_texts}
Please adhere to the following guidelines:
- Specify what the query is, and what the desired documents are.
- Your responses must be in Vietnamese language.
- Each retrieval task should cover a wide range of queries, and should not be too specific."""
TEMPLATE2 = """Your output tasks should always be a json object {'tasks': ['your_generated_task1', 'your_generated_task2', ...]}, with about 30 elements, and each element corresponds to a distinct retrieval task in one sentence. Do not explain yourself or output anything else. Be creative and follow the examples!"""


def sample_pool(n, pool):
    if n >= len(pool):
        return pool
    return list(np.random.choice(pool, size=n, replace=False))


def sample_seed(n, seed):
    return list(np.random.choice(seed, size=n, replace=False))


def get_response(examples):
    while True:
        try:
            sleep(4)
            prompt = TEMPLATE.format(
                demonstration_texts="\n".join([f"- {e}" for e in examples])
            ) + "\n" + TEMPLATE2
            response = g4f.ChatCompletion.create(
                provider=g4f.Provider.GeekGpt,
                model=g4f.models.gpt_4,
                messages=[{"role": "user", "content": prompt}],
            )
            return response
        except Exception as e:
            print(e)
            print("Request failed")
            sleep(15)


def extract_json(response):
    start = response.find("{")
    end = response.find("}") + 1
    if start == -1 or end == -1:
        return None
    try:
        return eval(response[start:end])
    except Exception as e:
        return None


def remove_duplicates(l, threshold=80):
    final = []
    for i in l:
        can_add = True
        for j in final:
            score = fuzz.ratio(i, j)
            if score >= threshold:
                can_add = False
                break
        if can_add:
            final.append(i)
    print(
        f"Removed {len(l) - len(final)} similar questions from total {len(l)} questions."
    )
    return final


def generate_questions(
    n_rounds=20,
    seed_path="./seed.txt",
    pool_path="./pool.txt",
    threshold=80,
    n_seed=2,
    n_pool=2,
):
    for i in range(n_rounds):
        with open(pool_path, "r") as f:
            question_pool = f.read().splitlines()

        with open(seed_path, "r") as f:
            seed_tasks = f.read().splitlines()

        print(f"Round {i+1}")
        seed = sample_seed(n_seed, seed_tasks)
        pool = sample_pool(n_pool, question_pool)
        response = get_response(seed + pool)
        js = extract_json(response)
        if js is None:
            print("No json found")
            continue
        print(js)
        with open(pool_path, "a") as f:
            for task in js["tasks"]:
                f.write(str(task) + "\n")

        if (i + 1) % 5 == 0:
            with open(pool_path, "r") as f:
                lines = [line.strip() for line in f.read().splitlines()]
            lines = remove_duplicates(lines, threshold=threshold)

            with open(pool_path, "w") as f:
                for line in lines:
                    f.write(str(line) + "\n")

In [ ]:
generate_questions()

In [ ]:
import random
from time import sleep
import g4f
import json
from tqdm import tqdm

pool_path = "./tasks/pool.txt"
with open(pool_path, "r") as f:
    tasks = [line.strip() for line in f.read().splitlines()]

TEMPLATE = """You have been assigned a retrieval task: {task}
Your mission is to write one text retrieval example for this task in JSON format. The JSON object must contain the following keys:
- "user_query": a string, a random user search query specified by the retrieval task.
- "positive_document": a string, a related document for the user query.
- "relevant_document": a string, a document that is NOT related to the user query but can still be in the same topic.
PLEASE adhere to the following guidelines:
- The "user_query" should be a {query_type}, at least {query_len} words long, and diverse in topic.
- All documents should be at least {doc_len} words long.
- Both the query and documents should be in Vietnamese.
- DO NOT repeat the words in the user_query in the positive_document, the content needs to be retrieved should be hidden inside the positive_document, and SHOULD NOT be too obvious or easy to find.
Your output must always be a JSON object only, do not explain yourself or output anything else. Be creative and follow the rules!"""

query_lens = ["1 to 5", "5 to 10", "10 to 15", "15 to 20"]
doc_lens = ["80", "120", "150", "200", "250", "300"]
query_types = ['question', 'search query']

def extract_json(response):
    start = response.find("{")
    end = response.find("}") + 1
    if start == -1 or end == -1:
        return None
    try:
        return eval(response[start:end])
    except Exception as e:
        return None


def get_response(task):
    while True:
        try:
            sleep(3)
            prompt = TEMPLATE.format(
                task=task,
                query_len=random.choice(query_lens),
                doc_len=random.choice(doc_lens),
                query_type=random.choice(query_types),
            )
            response = g4f.ChatCompletion.create(
                provider=g4f.Provider.GeekGpt,
                model=g4f.models.gpt_4,
                messages=[{"role": "user", "content": prompt}],
            )
            return response
        except Exception as e:
            print("Request failed")
            sleep(15)

# data = []
# start = 0
# end = start + 100
# for i, task in tqdm(enumerate(tasks[start:end], start=start)):
#     result = {"task": task, "generated": get_response(task)}
#     data.append(result)
#     if (i + 1) % 20 == 0:
#         with open(f"./tasks3/to_{i}.json", "w", encoding="utf-8") as f:
#             json.dump(data, f)
#         data = []

get_response(random.choice(tasks))

In [ ]:
eval("""{\n  "user_query": "Cách nấu mì ý bằng nồi cơm điện và nhận xét từ người thực hiện",\n  "positive_document": "Món ăn ngon và đơn giản với nồi cơm điện là mì ý. Để nấu mì ý bằng nồi cơm điện, đầu tiên hãy chuẩn bị mì và nước sôi. Cho mì vào nước sôi, chờ mì chín mềm rồi hãy nhấc ra. Trong khi đó, hãy chuẩn bị sốt bò bằng cách xào bò với cà chua, hành tỏi và các loại gia vị khác. Khi mì đã chín và sốt bò đã sẵn sàng, hãy trộn đều mì với sốt bò. Bạn sẽ có ngay một bữa ăn ngon miệng và dễ nấu bằng nồi cơm điện."\n\n  "relevant_document": "Làm thế nào để nấu mì ý ngon mắt với lò nướng và đánh giá từ người thử nghiệm. Mì ý có thể được nấu bằng nhiều cách khác nhau, và một trong những phương pháp phổ biến là sử dụng lò nướng. Để nấu mì ý với lò nướng, bạn cần chuẩn bị những nguyên liệu như mì, sốt bò, phô mai và các loại rau củ. Đầu tiên, nấu mì theo hướng dẫn, sau đó trang trí mì với sốt bò, phô mai và rau củ theo khẩu vị cá nhân. Đặt mì vào lò nướng và nướng cho đến khi phô mai tan chảy và mì có lớp vỏ giòn. Bạn sẽ có một bữa ăn mì ý tuyệt vời, và nhận xét từ những người đã thử nghiệm cho thấy đây là một cách nấu mì ý ngon và lạ miệng." \n}""")

In [7]:
import random

passage = """Title: Đài Phát thanh - Truyền hình Vĩnh Long Chương trình "Câu chuyện cuộc sống" với chủ đề "Game nhiệm vụ cướp của giết người - Giải trí hay cổ xúy phạm pháp" phát sóng trên THVL1 ngày 2 tháng 8 năm 2018 cho rằng những tựa game sinh tồn, trong đó có PlayerUnknown's Battleground (hay còn được gọi tắt là PUBG) và Minecraft là "có khuynh hướng bạo lực". Sau khi video lên sóng đã nhận rất nhiều chỉ trích từ cộng đồng game thủ. Sau đó, Đài đã tự xóa video. Tuy nhiên, vào tháng 5 năm 2019, Đài lại đăng tiếp một video với tiêu đề "Nghiện game online còn khó cai hơn nghiện ma túy". Video này cũng bị xóa sau khi nhận số lượng dislike lớn trên YouTube."""

TEMPLATE = """You have been given a passage in Vietnamese:
==PASSAGE==
{passage}
==YOUR TASK==
Your task is to write two type of queries for the passage in JSON format MUST containing the TWO following keys:
- "relevant_{query_type}"
- "irrelevant_{query_type}"
PLEASE adhere to the following guidelines:
- The relevant_{query_type} should be a {query_type} that is related to the passage.
- The irrelevant_{query_type} should be a {query_type} that is NOT related to the passage but can still be in the same topic.
- Try to combine the information in the passage to form the relevant_{query_type}.
- Do not repeat the words in the passage in the relevant_{query_type}, the content needs to be retrieved should be hidden inside the relevant_{query_type}, and SHOULD NOT be too obvious to spot.
- Generated text must be in Vietnamese.
- All the {query_type}s should be at least {query_len} words long.
Be creative and follow the rules!"""

query_lens = ["7 to 10", "10 to 15", "15 to 20", "20 to 25"]
query_types = ["question", "claim"]


def get_response(passage=passage, sleep_time=3):
    print("Sleeping for", sleep_time, "seconds")
    sleep(sleep_time)
    while True:
        try:
            prompt = TEMPLATE.format(
                passage=passage,
                query_len=random.choice(query_lens),
                query_type=random.choice(query_types),
            )
            response = g4f.ChatCompletion.create(
                provider=g4f.Provider.GeekGpt,
                model=g4f.models.gpt_4,
                messages=[{"role": "user", "content": prompt}],
            )
            return response
        except Exception as e:
            print(e)
            print("Request failed")
            sleep(random.randint(10, 15))


get_response(passage)

Sleeping for 3 seconds


'```json\n{\n  "relevant_question": "Lý do Đài Phát thanh - Truyền hình Vĩnh Long xóa video sau khi nhận chỉ trích từ cộng đồng game thủ là gì?",\n  "irrelevant_question": "Làm thế nào để chăm sóc cây cảnh trong môi trường có khí hậu lạnh?"\n}\n```'

In [10]:
import concurrent.futures

num_threads = 10

with concurrent.futures.ThreadPoolExecutor(max_workers=num_threads) as executor:
    results = list(executor.map(get_response, [passage] * num_threads, [4*i for i in range(num_threads)]))

Sleeping forSleeping for 4 seconds
 0 seconds
Sleeping for 8 seconds
Sleeping for 12 seconds
Sleeping for 16 seconds
Sleeping for 20 seconds
Sleeping for 24 seconds
Sleeping for 28 seconds
Sleeping for 32 seconds
Sleeping for 36 seconds
429 Client Error: Too Many Requests for url: https://ai.fakeopen.com/v1/chat/completions
Request failed


In [4]:
results

['```json\n{\n  "relevant_question": "Tại sao Đài Phát thanh - Truyền hình Vĩnh Long đã nhận nhiều chỉ trích từ cộng đồng game thủ sau khi phát sóng chương trình với chủ đề \'Game nhiệm vụ cướp của giết người\'?",\n  "irrelevant_question": "Làm thế nào để nâng cao kỹ năng chơi game online và tránh bị nghiện, không liên quan đến việc Đài Phát thanh - Truyền hình Vĩnh Long xóa video về \'Nghiện game online còn khó cai hơn nghiện ma túy\'?"\n}\n```',
 '{\n  "relevant_claim": "Sự chỉ trích của cộng đồng game thủ đối với Đài Phát thanh - Truyền hình Vĩnh Long về việc xóa video về game PUBG và Minecraft.",\n  "irrelevant_claim": "Nguồn cung cấp thông tin về việc làm thủ tục hành chính tại thành phố Vĩnh Long và các kênh thông tin liên quan."\n}',
 '```json\n{\n  "relevant_question": "Lý do Đài Phát thanh - Truyền hình Vĩnh Long xóa video về tựa game PUBG và Minecraft sau khi nhận chỉ trích từ cộng đồng game thủ là gì?",\n  "irrelevant_question": "Làm thế nào để nâng cao kỹ năng chơi game tro